## Практическое задание к уроку 7 (Сверточные нейронные сети для анализа текста)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
data_path = '/content/drive/MyDrive/Colab Notebooks/geekbrains natural language processing/homework7'

In [3]:
!pip install stop_words
!pip install pymorphy2

### Задание

### Берем отызывы за лето (из архива с материалами или предыдущего занятия)

In [4]:
import pandas as pd
import os

In [5]:
df = pd.read_excel(os.path.join(data_path, 'reviews.xls'))
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


### Препроцессинг

In [6]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [7]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)
df['Rating'] = df['Rating'].apply(lambda x: f'{x}')

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Rating', 'Date']), df['Rating'], test_size=0.33, random_state=42)

In [10]:
train_corpus = " ".join(X_train["Content"])
train_corpus = train_corpus.lower()

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
tokens_filtered = [word for word in tokens if word.isalnum()]
tokens_filtered[:10]

['наконецтый',
 'исправить',
 'чушь',
 'снеоргинальный',
 'прошивка',
 'приложение',
 'удобно',
 'пользоваться',
 'удобно',
 'использование']

In [13]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [14]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'10': 190,
 '2': 88,
 '3': 120,
 '4': 153,
 '5': 64,
 'meizu': 65,
 'root': 62,
 'андроид': 167,
 'антивирус': 16,
 'банк': 17,
 'банкомат': 157,
 'безопасность': 129,
 'большой': 123,
 'быстро': 11,
 'быстрый': 130,
 'ваш': 101,
 'ввести': 133,
 'ввод': 72,
 'вводить': 31,
 'вернуть': 166,
 'версия': 76,
 'видеть': 161,
 'вирус': 112,
 'возможность': 55,
 'войти': 71,
 'вообще': 83,
 'встроить': 179,
 'вход': 18,
 'выдавать': 110,
 'вылетать': 50,
 'выходить': 118,
 'главное': 160,
 'грузиться': 142,
 'давать': 165,
 'дать': 138,
 'делать': 57,
 'дело': 156,
 'деньга': 36,
 'добавить': 121,
 'довольный': 37,
 'долгий': 169,
 'дом': 117,
 'доступ': 75,
 'доступно': 191,
 'другой': 128,
 'ждать': 124,
 'зависать': 185,
 'загружаться': 162,
 'загрузка': 172,
 'зайти': 48,
 'замечательный': 127,
 'заново': 199,
 'запуск': 95,
 'запускаться': 116,
 'заходить': 63,
 'звезда': 151,
 'знать': 144,
 'идти': 141,
 'изз': 89,
 'инициализация': 148,
 'интернет': 86,
 'интерфейс': 136,
 'иня': 58

In [15]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []

    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen - len(result))

    return padding + result[-maxlen:]

In [16]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train["Content"]], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test["Content"]], dtype=np.int32)

In [17]:
X_train, X_test

(array([[  0,   0,   0, ...,   1,   2,  15],
        [  0,   0,   0, ...,   0,   2, 181],
        [  0,   0,   0, ...,   0,   0,   5],
        ...,
        [  0,   0,   0, ..., 164,  27,  84],
        [  0,   0,   0, ...,   0,   0,  20],
        [  0,   0,   0, ...,   0, 113,   5]], dtype=int32),
 array([[ 0,  0,  0, ...,  0,  0,  5],
        [ 0,  0,  0, ...,  0,  6,  2],
        [ 0,  0,  0, ...,  1, 14, 48],
        ...,
        [ 0,  0,  0, ...,  1,  2, 11],
        [ 0,  0,  0, ..., 15,  1, 42],
        [ 0,  0,  0, ..., 32,  1, 91]], dtype=int32))

In [18]:
import numpy as np
import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [19]:
y_train.shape

(13841,)

In [20]:
num_classes = len(df['Rating'].value_counts()) + 1
num_classes

6

In [21]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

1. Создаем и учим модель с обычным Embedding

In [22]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
!pip install tensorflow==2.5.0

In [25]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 21s 163ms/step - loss: 1.4971 - accuracy: 0.6886 - val_loss: 1.0693 - val_accuracy: 0.7011


In [26]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 0s 34ms/step - loss: 1.0226 - accuracy: 0.7125


Test score: 1.0225632190704346
Test accuracy: 0.7125256657600403


In [27]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 34ms/step



2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

2. Создаем и учим модель с предзагруженным Embedding

грузим с сайта https://rusvectores.org/ru/  
модель ruwikiruscorpora_tokens_elmo_1024_2019

In [28]:
!pip install --upgrade simple_elmo

In [29]:
from simple_elmo import ElmoModel
model_elmo = ElmoModel()
model_elmo.load(os.path.join(data_path, 'model'))

2021-07-29 17:42:09,138 : INFO : Loading model from /content/drive/MyDrive/Colab Notebooks/geekbrains natural language processing/homework7/model...
2021-07-29 17:42:09,151 : INFO : We will cache the vocabulary of 100 tokens.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


'The model is now loaded.'

In [ ]:
vectors = model_elmo.get_elmo_vectors(df['Content'])

на строчке выше отваливаюсь по RAM в colab, попробую решить вопрос на локальной машине

In [ ]:
import pickle
with open('embeddings.pickle', 'wb') as f:
    pickle.dump(vectors, f)